# 연습문제2 — 한국은행(ECOS) 데이터 실습

> **준비**: **`api_key`**만 본인 키 준비하세요.

---

## 문제 개요
한국은행 ECOS API에서 재무상태표(stat_code=`501Y001`)의 **전산업(ZZZ00)** 항목을 내려받아 전처리(결측치 처리 포함) 후 시계열 시각화와 간단한 수치분석을 수행하시오. 분석 기간은 **2009년부터 2023년**까지입니다.

---

## 고정 파라미터 
- **통계표 코드 (stat_code)**: `501Y001` (재무상태표 — 연간)  
- **주기 (frequency)**: `A` (연간, Yearly) — 연도 단위(YYYY)  
- **세부항목 (item_code1)**: `ZZZ00` (전산업 / All industries)  
- **분석 기간**: `2009` (2009년) 부터 `2023` (2023년) 까지  
- **요청 URL 예시**:  
  ```
  https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/9999/501Y001/A/2009/2023/ZZZ00
  ```

---

## 제출물 
1. **전처리 결과 CSV**  
   - 파일명: `ecos_501Y001_ZZZ00_annual_cleaned.csv`  
   - 포함 컬럼(권장): `TIME`, `year`, `date`, `value`, `value_filled`, `ma_3`, `pct_yoy`  
   - `date`는 `YYYY-01-01` 형식의 datetime으로 표기하고, 인덱스로 사용할 수 있습니다.
2. **작성된 파이썬 코드**  
   - 연습문제2.ipynb

---

## 전처리 규칙 (명확히 지킬 것)
1. **시간 변환**  
   - API의 `TIME` 값은 연도형식(`YYYY`)입니다. `year` 컬럼(정수)을 만들고 `date` 컬럼은 `YYYY-01-01` 형식의 datetime으로 생성하세요.  
   - `date`를 DataFrame 인덱스로 설정하면 시계열 리샘플링/정렬이 용이합니다.
2. **값 처리**  
   - 값 컬럼(예: `DATA_VALUE` 형태)을 찾아서 문자열 내 쉼표(,)를 제거한 후 숫자형으로 변환하세요.  
   - 변환 불가능한 값은 `NaN`으로 처리합니다.
3. **결측치 처리 (명시적 규칙)**  
   - 모든 `NaN`은 **해당 열의 평균(mean)** 으로 대체합니다. 대체 후 새 컬럼명으로 `value_filled`를 사용하세요.  
   
4. **보조 컬럼 생성**  
   - `ma_3`: `value_filled`의 3년 이동평균 (pandas `rolling(window=3, min_periods=1).mean()`)  
   - `pct_yoy`: `value_filled`의 전년대비(%) = `(t - t-1) / t-1 * 100` (`pct_change(periods=1) * 100`)
5. **정렬 및 저장**  
   - 인덱스는 `date`로 오름차순 정렬(`sort_index()`)하고, 최종 DataFrame을 지정된 CSV 파일명으로 저장하세요 (`utf-8-sig` 권장).

---

## 분석/시각화 요구사항
### 1) Time series plot (필수)  
- **내용**: `value_filled`(원본 결측 대체값)와 `ma_3`(3-year moving average)를 같은 플롯에 겹쳐 표시하세요.  
- **레이블(영어 권장)**: Title = `"Balance Sheet - All Industries (Annual)"`, x = `"Year"`, y = `"Amount (unit as provided by API)"`  
- **파일명**: `timeseries_501Y001_ZZZ00.png`  
- **요구사항**: 범례(legend), 그리드(grid) 포함. 저장시 `dpi=150` 권장.

### 2) Annual bar chart (필수)  
- **내용**: `value_filled`를 연간 평균(aggregation)으로 재표집하여 막대그래프로 표시하세요. (pandas `resample('YE').mean()` 권장)  
- **레이블(영어 권장)**: Title = `"Annual Average: All Industries"`, x = `"Year"`, y = `"Average Amount"`  
- **파일명**: `annual_bar_501Y001_ZZZ00.png`  
- **요구사항**: x축 라벨 회전(`rotation=45`)으로 가독성 확보.

### 3) 수치 분석(필수)  
- **최신 연도값**: 가장 최신 연도의 `value_filled` 값을 제시.  
- **전년값**: 최신 연도보다 1년 전의 값(존재하지 않으면 `NaN`) 제시.  
- **전년대비(%)**: 최신 YoY (%) 계산(존재하지 않으면 `NaN`).  
- **3년 평균 비교**:  
  - `Recent 3-year mean`: 2021–2023 평균(`value_filled`)  
  - `Previous 3-year mean`: 2018–2020 평균(`value_filled`)  
  - 변화율(%) = `(recent - previous) / previous * 100` (previous가 0이면 `NaN` 처리)

---

## 결과 참고

=== Summary ===
Latest year: 2023, value = 702469172.445

1 year before: 2022, value = 670244905.939

YoY % (latest) = 4.807834602032642

Recent 3-year mean (2021-2023) = 661989063.634

Previous 3-year mean (2018-2020) = 580592420.444

3-year change rate (%) = 14.020



<img src="https://imgur.com/7zUz937.png" width="80%">

<img src="https://imgur.com/C44AGee.png" width="80%">


In [ ]:
import requests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,5)
sns.set_style('whitegrid')
